In [27]:
import methylize
import methylcheck
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [28]:
betas, meta = methylcheck.load_both('../data/asthma') #load in the beta values and metadata
betas = betas.T

INFO:methylcheck.load_processed:Found several meta_data files; attempting to match each with its respective beta_values files in same folders.
INFO:methylcheck.load_processed:Multiple meta_data found. Only loading the first file.
INFO:methylcheck.load_processed:Loading 40 samples.
Files: 100%|██████████| 1/1 [00:00<00:00, 10.86it/s]
INFO:methylcheck.load_processed:loaded data (485512, 40) from 1 pickled files (0.108s)
INFO:methylcheck.load_processed:Transposed data and reordered meta_data so sample ordering matches.
INFO:methylcheck.load_processed:meta.Sample_IDs match data.index (OK)


In [29]:
betas = betas.dropna(how='any', axis='columns')
betas = betas.sample(5000, axis = 1)

In [30]:
betas.isna().sum().sum()

0

In [31]:
merge = betas.merge(meta[['Sample_ID', 'disease', 'age']], left_index=True, right_on='Sample_ID').drop(columns='Sample_ID')
merge.head()

,cg23079252,cg23487517,cg12446209,cg07764932,cg03855338,cg25867569,cg00057476,cg01164326,cg25470731,cg14260418,...,cg07108083,cg06664668,cg11844358,cg27543320,cg20969543,cg04471151,cg23905164,cg03535239,disease,age
20,0.035,0.229,0.788,0.529,0.613,0.957,0.689,0.034,0.020,0.959,...,0.856,0.025,0.987,0.368,0.973,0.069,0.190,0.938,Healthy,65
38,0.043,0.139,0.824,0.192,0.865,0.968,0.533,0.029,0.021,0.961,...,0.931,0.028,0.989,0.510,0.965,0.134,0.200,0.929,Asthmatic,19
37,0.041,0.140,0.698,0.459,0.740,0.967,0.597,0.031,0.024,0.964,...,0.930,0.030,0.990,0.376,0.967,0.089,0.124,0.964,Asthmatic,14
32,0.038,0.119,0.767,0.206,0.757,0.969,0.526,0.035,0.022,0.964,...,0.888,0.030,0.990,0.450,0.970,0.087,0.236,0.940,Asthmatic,15
23,0.053,0.336,0.690,0.374,0.796,0.961,0.708,0.035,0.025,0.961,...,0.885,0.033,0.985,0.344,0.975,0.068,0.229,0.913,Asthmatic,8


In [32]:
lin = LinearRegression().fit(merge.drop(columns=['disease', 'age']).to_numpy(),merge.age)

In [33]:
X = np.asarray(merge.drop(columns=['disease', 'age']))
Y = np.asarray(merge.age).astype(float)

In [34]:
from sklearn.feature_selection import f_regression

f = f_regression(X, Y)
pvals = f[1]

In [35]:
test_results2 = methylize.diff_meth_pos(betas, merge.age, regression_method="linear", export=False)
test_results2

  0%|          | 0/5000 [00:00<?, ?it/s]

,Coefficient,StandardError,PValue,95%CI_lower,95%CI_upper,Rsquared,FDR_QValue
cg12613632,-6.860594e-03,0.000776,9.323498e-11,-0.820788,-0.819793,6.728778e-01,4.661749e-07
cg13800769,-4.029023e-03,0.000506,1.278717e-09,-0.791076,-0.790333,6.252137e-01,3.196794e-06
cg09363076,4.935973e-03,0.000658,5.293160e-09,0.771864,0.772905,5.965783e-01,8.821933e-06
cg02674839,-2.516179e-03,0.000348,1.214597e-08,-0.761115,-0.760541,5.788595e-01,1.518246e-05
cg02155957,-2.609670e-03,0.000380,3.781235e-08,-0.744245,-0.743580,5.534061e-01,3.781235e-05
...,...,...,...,...,...,...,...
cg24675879,-8.962639e-08,0.000095,9.992483e-01,-0.000339,0.000031,2.366420e-08,9.998483e-01
cg26821634,6.663195e-07,0.000647,9.991839e-01,-0.001101,0.001435,2.789435e-08,9.998483e-01
cg13296238,-9.652294e-08,0.000593,9.998711e-01,-0.001189,0.001137,6.964350e-10,9.999051e-01
cg08678154,3.503591e-08,0.000293,9.999051e-01,-0.000554,0.000593,3.768331e-10,9.999051e-01


In [36]:
pvals = ['%.10f' % i for i in pvals]
methp = ['%.10f' % i for i in test_results2.PValue]

In [37]:
same = 0
for i, j in zip(sorted(pvals), sorted(methp)):
    if i == j:
        same += 1
same

4996

In [51]:
Y = merge['disease'].replace({'Healthy':0, "Asthmatic":1}).astype(float)
from sklearn import linear_model
import numpy as np
import scipy.stats as stat

class LogisticReg:
    """
    Wrapper Class for Logistic Regression which has the usual sklearn instance 
    in an attribute self.model, and pvalues, z scores and estimated 
    errors for each coefficient in 
    
    self.z_scores
    self.p_values
    self.sigma_estimates
    
    as well as the negative hessian of the log Likelihood (Fisher information)
    
    self.F_ij
    """
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        #### Get p-values for the fitted model ####
        denom = (2.0*(1.0+np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0]/sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values
        
        self.z_scores = z_scores
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.F_ij = F_ij
        return p_values

l = LogisticReg()
results = l.fit(X, Y)

/Users/jaredmeyers/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [52]:
results

[0.9999999999951651,
 nan,
 0.9999999974415138,
 0.9999999990650362,
 0.9999999618217957,
 0.9999999998845046,
 0.9999999999212826,
 nan,
 nan,
 0.9999999993016656,
 0.9999999994220206,
 nan,
 nan,
 nan,
 0.9999999990429657,
 nan,
 0.9999999983384904,
 0.9999999993335464,
 0.9999999999946304,
 0.9999999999128786,
 nan,
 0.9999999989694706,
 nan,
 0.9999999989790546,
 nan,
 0.999999999982477,
 nan,
 nan,
 0.9999999999979937,
 0.9999999988628754,
 nan,
 nan,
 nan,
 0.9999999996602567,
 nan,
 0.9999999999797308,
 nan,
 0.9999999994334969,
 nan,
 0.9999999989457483,
 0.9999999982322962,
 0.9999999995949681,
 0.9999999999694381,
 0.9999999999993321,
 nan,
 0.99999999932079,
 nan,
 0.9999999999941039,
 0.999999999948677,
 0.9999999991086113,
 nan,
 nan,
 nan,
 0.9999999952798114,
 0.9999999992021632,
 0.9999999996206266,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.9999999998821426,
 0.9999999999834438,
 nan,
 nan,
 0.9999999791660164,
 nan,
 0.9999999999148973,
 0.9999999999982817,
 0.9999999999755259,

In [ ]:
methylize.diff_meth_pos(betas, Y, regression_method='logistic')

  0%|          | 0/100 [00:00<?, ?it/s]

,Coefficient,StandardError,PValue,95%CI_lower,95%CI_upper,FDR_QValue
cg01471428,-0.315374,0.352441,0.370880,-1.006146,0.375398,0.494681
cg24583481,-0.336419,0.357152,0.346219,-1.036425,0.363587,0.494681
cg07544102,-0.441948,0.448897,0.324860,-1.321771,0.437874,0.494681
cg01849531,-0.323910,0.346567,0.349982,-1.003168,0.355348,0.494681
cg12209743,-12.822300,10.142822,0.206167,-32.701866,7.057267,0.494681
...,...,...,...,...,...,...
cg09226262,-0.288546,0.732426,0.693611,-1.724076,1.146983,0.722511
cg26639809,-0.172671,0.464008,0.709796,-1.082111,0.736768,0.731749
cg18118249,0.180648,0.754843,0.810859,-1.298817,1.660113,0.827407
cg10025432,-0.163505,0.779957,0.833954,-1.692192,1.365183,0.842378
